In [1]:
from __future__ import print_function, division
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, sampler

from torchvision import transforms, utils, models
from torch.optim import lr_scheduler

import numpy as np
from skimage import io, transform
import pandas as pd
import matplotlib.pyplot as plt

import time
import copy

from torch.nn.functional import interpolate

import encoding

In [36]:
# Set up hyperparameters and train settings
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
MIN_SAMPLES_PER_CLASS = 50
BATCH_SIZE = 512
LEARNING_RATE = 1e-3
LR_STEP = 3
LR_FACTOR = 0.5
# NUM_WORKERS = multiprocessing.cpu_count()
MAX_STEPS_PER_EPOCH = 15000
NUM_EPOCHS = 2 ** 32
LOG_FREQ = 500
NUM_TOP_PREDICTS = 20
TIME_LIMIT = 9 * 60 * 60

dtype = torch.cuda.FloatTensor
device = torch.device("cuda:0")

In [40]:
# Loading Train Dataset

class LandmarksDataset(Dataset):
    """Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0]) #INCORRECT, NEEDS TO FIT TRAIN FOLDER NAMING
        img_name += '.jpg'
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 2]
        sample = {'image': image, 'class': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample
    

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'landmarks': torch.from_numpy(landmarks)}
    

landmarks_dataset = LandmarksDataset(csv_file = 'train.csv',
                                   root_dir = 'train/',
                                   transform = ToTensor()
                                  )

#Confirm shapes and labels are of correct form
for i in range(len(landmarks_dataset)):
    sample = landmarks_dataset[i]

    print(i, sample['image'].size(), sample['landmarks'])

    if i == 3:
        break
        
class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples
        
dataloaders = DataLoader(landmarks_dataset, batch_size=4,
                        shuffle=True, num_workers=4)

FileNotFoundError: [Errno 2] No such file or directory: 'train/97c0a12e07ae8dd5.jpg'

In [ ]:
# Import pre-trained FCN ResNet-50 model

# FCN_model = models.segmentation.fcn_resnet50(pretrained=True, 
#                                                          progress=True, 
#                                                          num_classes=200000)

FCN_model = encoding.models.get_model('FCN_ResNet50_ADE', pretrained=True)

# print(list(FCN_model.children()))

# print(len(list(list(FCN_model.children())[0])))

for param in FCN_model.parameters():
    param.requires_grad = False

FCN_model.fc = nn.Linear(1024, 2)

#FCN_model = FCN_model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_FCN = optim.SGD(FCN_model.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_FCN, step_size=7, gamma=0.1)

# print(newmodel)


In [41]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

In [39]:
FCN_model = train_model(FCN_model, criterion, optimizer_FCN, exp_lr_scheduler,
                       num_epochs=20)

Epoch 0/19
----------


NameError: name 'dataloaders' is not defined

In [ ]:
model = nn.Sequential(
    nn.Conv2d(in_channels=, out_channels=224, kernel_size=5, stride=1, padding=0),
    nn.ReLU(), 
)

In [ ]:
# Prediction
a = torch.Tensor(range(224))
model(a)